In [1]:
cd C:\Users\samyu\9_20_22 data\updated files 9_27_22\autoscaled training and test set\wo braca

C:\Users\samyu\9_20_22 data\updated files 9_27_22\autoscaled training and test set\wo braca



## Machine learning pipeline
1. [Pearson correlation and t-test](#1)
2. [Random forests feature selection between imbalanced OC and non-OC samples](#2)
3. [Balance training data with SMOTE](#3)
4. [Feature selection for balanced training set](#4)
5. [Early stage samples from test set](#5)






In [2]:
#import libraries

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import style

plt.style.use('ggplot')
import scipy

import seaborn as sns; sns.set(style='white')

import warnings
warnings.filterwarnings('ignore')

%config InlineBackend.figure_format = 'retina'

sns.set_context('talk', font_scale=0.8)
import statsmodels as sms
from statsmodels.stats import multitest
from statistics import mean
np.random.seed()

### Code to print versions of packages used 

In [6]:
# reference https://stackoverflow.com/questions/40428931/package-for-listing-version-of-packages-used-in-a-jupyter-notebook 

import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]
            
        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to add
        # exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]
            
        yield name
imports = list(set(get_imports()))

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names 
# of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

statsmodels==0.12.2
seaborn==0.11.2
scipy==1.7.1
scikit-learn==1.1.2
pandas==1.3.4
numpy==1.20.3
matplotlib==3.4.3
imblearn==0.0


#### Import dataframe with all annotated features and samples 

In [20]:
#import dataframe with OC and non-oc samples and all annotated features 
df_ = pd.read_excel("without braca.xlsx", sheet_name ='Sheet1')
#df_.head()

In [21]:
#assign class and features 
other_ = df_[(df_['class'] == 'other')]
OC_ = df_[(df_['class'] == 'OC')]

#list of features
features_ = df_.drop(['sample','class'],axis = 1).columns

<a id="1"></a> 
### Pearson Corrleation and t-test

In [22]:
#remove one of the correlated features
correlated_features_ = set()
correlation_matrix = df_[features_].astype(float).corr() 

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.85:
            colname = correlation_matrix.columns[i]
            correlated_features_.add(colname)

In [23]:
corr_feat_ = df_[features_].drop(correlated_features_, axis=1)
corr_feat_.astype(float);

In [24]:
corr_feat_list_ = corr_feat_.columns
len(corr_feat_list_)

789

#### Import Imbalanced training set 
OC = 144 and non-OC = 83

In [25]:
cd C:\Users\samyu\9_20_22 data\updated files 9_27_22\autoscaled training and test set

C:\Users\samyu\9_20_22 data\updated files 9_27_22\autoscaled training and test set


In [26]:
#training 144 OC and 83 non-oC
#import dataframe with OC and non-oc samples and all annotated features
df = pd.read_excel("training set initial.xlsx", sheet_name ='training set initial wo BRACA')
df.head()

,sample,class,feat1Car(10:0)_P2,feat2Car(12:0)_P3,feat3Car(12:1)_P4,feat4Car(13:0)_P5,feat5Car(13:1)_P6,feat6Car(14:0)_P7,feat7Car(14:2)_P8,feat8Car(16:0)_P9,...,feat992SM(d44:2) M + CH3COOH,feat993SM(t38:0)_or_SM(d38:0-OH) M + CH3COOH,feat994SM(t39:0)_or_SM(d39:0-OH) M + CH3COOH,feat995SM(t40:0)_or_SM(d40:0-OH) M + CH3COOH,feat996SM(t42:0)_or_SM(d42:0-OH) M + CH3COOH,feat997SM(t42:1)_or_SM(d42:1-OH) M + CH3COOH,feat998sphingosine-1P (C18),feat999Sulfatide(d34:1),feat1000Sulfatide(d34:1-OH),feat1001Sulfatide(d43:3)
0,CC20,other,5.927164e+06,1.978869e+06,2.439032e+06,6.037485e+05,4.108407e+05,1.228990e+06,1.702951e+06,8.036638e+06,...,2.438449e+06,3.708847e+06,2.287170e+07,1.224911e+06,367801.650540,8.762374e+06,1.563349e+06,377895.733114,5.941570e+05,110571.416746
1,BOT2,other,9.526528e+05,4.222392e+05,5.341055e+05,1.408429e+06,1.006750e+06,9.830583e+05,5.074331e+05,8.525652e+06,...,3.348559e+06,4.297437e+06,2.970096e+07,9.510993e+05,273309.524313,3.816278e+06,1.802775e+06,943402.553774,1.264786e+06,252801.935443
2,BOT1,other,7.738506e+05,5.564775e+05,7.315308e+05,4.631695e+05,2.332695e+05,5.129053e+05,8.420001e+05,4.594929e+06,...,2.322826e+06,3.552778e+06,1.762786e+07,6.759426e+05,428429.952705,3.128441e+06,1.201455e+06,626345.058651,1.055082e+06,175852.080971
3,BOT13,other,1.594204e+06,3.987027e+05,7.283001e+05,2.735521e+05,1.726858e+05,4.472934e+05,4.827808e+05,8.560225e+06,...,1.895642e+06,4.271366e+06,1.959419e+07,1.383601e+06,480731.001943,9.496677e+06,1.990074e+06,502025.837365,7.359521e+05,184892.677773
4,BOT14,other,1.720800e+06,8.881493e+05,1.397212e+06,7.625247e+05,3.677223e+05,9.565847e+05,1.160348e+06,8.690129e+06,...,1.386838e+06,3.645983e+06,7.062562e+06,8.949550e+05,462049.272721,4.321886e+06,1.087197e+06,669113.167281,9.086103e+05,253598.697048


In [27]:
#assign class and features 
other = df[(df['class'] == 'other')]
OC = df[(df['class'] == 'OC')]

#list of features
features = df.drop(['sample','class'],axis = 1).columns

In [28]:
print('OC =', len(OC))
print('Other =', len(other))


OC = 144
Other = 83


In [29]:
def Ttest(metabolites, dfControl, dfTreat, alpha=0.05, var=True):
    ttest_dict = {}
    for metabolite in metabolites:
        statistic, pvalue =  scipy.stats.ttest_ind(dfControl[metabolite], 
                                               dfTreat[metabolite], 
                                               equal_var=var)
        ttest_dict[metabolite] = pvalue 
       
    ttest = pd.DataFrame.from_dict(ttest_dict, orient='index') # the dictionary in pandas df
    ttest_list=list(ttest_dict.values()) #pvalues of ttest result in a list
    reject, pval_corrected, _, _ = sms.stats.multitest.multipletests(ttest_list, 
                                                                 alpha=alpha, 
                                                                 method='fdr_bh')
    ttest_results = pd.DataFrame({'Annotation': metabolites, 'T-test p-value': ttest_list, 
                              'FDR p-value': pval_corrected, 'Reject H0': reject})
    Table = ttest_results.sort_values(by=['Annotation'])
    return Table

In [30]:
ttest_train = Ttest(corr_feat_list_, other, OC, alpha=0.05, var = True)
stat_sig = ttest_train.loc[ttest_train['T-test p-value'] <=0.05]

len(stat_sig)

237

In [31]:
stat_sig_feat = stat_sig['Annotation'].tolist()

In [32]:
feat_train = df.drop(['class','sample'],axis =1)  
#feat_train= df_train[feat_train]   #training data with all features
feat_train_sel = feat_train[stat_sig_feat]  #df of training samples with features selected after Ttest

In [33]:
dfgrp = df.filter(['class'], axis=1)
dfgroup = dfgrp['class'].map({'other': 0, 'OC': 1})
x= feat_train_sel #df with stat sig features
y= dfgroup.values #class as 0 or 1 for other and OC 

x_scaled = (x-x.mean(axis=0))/x.std(axis=0) #(autoscaled dataset)

<a id="2"></a>
### Random forests feature selection for imbalanced training data

In [3]:
import sklearn 
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier

# Import Random Forest classifier, sklearn metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# test train split and K-fold validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn import decomposition

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [35]:
def RF_selector(X, y):
    
    """
    Function: A random forest function
    X-Matrix is autoscaled. 
    
    Input: 
    X: X-Matrix
    Y: y matrix
    
    Output:Dataframe containing Gini index. 
    
    """
    
    # Instantiate random forests
    RFclf = RandomForestClassifier(
        n_estimators=100, 
        random_state=42
    )
    
    # Fit random forest
    RFclf.fit(X, y)
    
    # Gini index
    gini = RFclf.feature_importances_
    
    # Zip metabolite name to gini index. 
    list_of_tuples = list(
        zip(X.columns, 
            gini)
    )
    
    # Get dataframe for metabolites and Gini index score.
    df_feat = pd.DataFrame(
        list_of_tuples, 
        columns = ['Annotations', 'Gini_Index']
    )
    
    # Sort the dataframe
    df_feat = df_feat.sort_values(
        by='Gini_Index', 
        ascending=False
    )
    
    return df_feat

In [36]:
# features sorted with importance score (gini index)
RF_features = RF_selector(x_scaled, y)
RF_features # imbalanced dataset 

,Annotations,Gini_Index
219,feat931PC(O-40:7)>PC(O-18:1/22:6)_and_PC(O-22:...,0.018979
92,feat331PC(O-36:5)_P332,0.014903
90,feat32Cer(d34:1)>Cer(d18:1/16:0)_P33,0.013628
229,feat961PS(O-40:7)>PS(O-20:3/20:4) M + HCO2 + Na,0.013399
7,feat134LPC(14:0)>LPC(14:0/0:0)_and_LPC(0:0/14:...,0.012457
...,...,...
198,feat861PC(38:2)>PC(18:1_20:1) M + H2CO2,0.001018
157,feat569TG(51:1) M + NH3_P570,0.000950
65,feat279PC(42:2)_P280,0.000875
31,feat199PC(35:1)_P200,0.000831


In [37]:
#Select top 10 features 
RF_features_sel = RF_features['Annotations'][:10].tolist()
RF_features_sel = x_scaled[RF_features_sel]
RF_features_sel_list = RF_features_sel.columns
len(RF_features_sel_list)

10

### Features selected with imbalanced training set 

In [38]:
RF_features_sel_list

Index(['feat931PC(O-40:7)>PC(O-18:1/22:6)_and_PC(O-22:7/18:0) M + CH3COOH',
       'feat331PC(O-36:5)_P332', 'feat32Cer(d34:1)>Cer(d18:1/16:0)_P33',
       'feat961PS(O-40:7)>PS(O-20:3/20:4) M + HCO2 + Na',
       'feat134LPC(14:0)>LPC(14:0/0:0)_and_LPC(0:0/14:0)_P135',
       'feat758TG(58:8)>TG(18:0_20:4_20:4)_and_TG(58:8)>TG(18:0_18:2_22:6)_P759',
       'feat401PE(O-40:5)>PE(O-18:1/22:4)_P402',
       'feat895PC(O-34:2)>PC(O-16:0/18:2) M + CH3COOH',
       'feat960PS(40:5)>PS(18:0_22:5)', 'feat268PC(40:8)_P269'],
      dtype='object')

#### Import test set

In [39]:
df_test = pd.read_excel("training set initial.xlsx", sheet_name ='test set')
df_test

,sample,group_other,feat1Car(10:0),feat2Car(12:0)_P3,feat3Car(12:1)_P4,feat4Car(13:0)_P5,feat5Car(13:1)_P6,feat6Car(14:0)_P7,feat7Car(14:2)_P8,feat8Car(16:0)_P9,...,feat992SM(d44:2) M + CH3COOH,feat993SM(t38:0)_or_SM(d38:0-OH) M + CH3COOH,feat994SM(t39:0)_or_SM(d39:0-OH) M + CH3COOH,feat995SM(t40:0)_or_SM(d40:0-OH) M + CH3COOH,feat996SM(t42:0)_or_SM(d42:0-OH) M + CH3COOH,feat997SM(t42:1)_or_SM(d42:1-OH) M + CH3COOH,feat998sphingosine-1P (C18),feat999Sulfatide(d34:1),feat1000Sulfatide(d34:1-OH),feat1001Sulfatide(d43:3)
0,OC40,OC,2.090656e+06,9.431434e+05,1.205950e+06,4.814695e+05,3.145821e+05,1.060877e+06,1.035758e+06,6.926954e+06,...,4.910143e+05,6.077822e+05,1.261128e+06,1.200567e+05,19593.210754,6.057511e+05,2.448383e+06,59883.498847,6.643711e+04,61751.042629
1,OC47,OC,4.056112e+06,1.315961e+06,2.373766e+06,1.839025e+06,1.019359e+06,1.003641e+06,1.543958e+06,5.947698e+06,...,4.238941e+06,3.414771e+06,3.356157e+07,1.035535e+06,306841.376122,6.255257e+06,1.211042e+06,453299.297082,6.469592e+05,159336.614280
2,OC267,OC,1.312514e+06,5.927774e+05,1.160696e+06,8.910156e+05,3.605883e+05,8.008122e+05,6.856267e+05,8.002517e+06,...,3.923609e+06,3.978729e+06,2.801678e+07,1.552968e+06,349043.495932,7.499122e+06,1.724551e+06,837060.553859,1.296346e+06,236523.800434
3,OC65,OC,1.582785e+06,9.058238e+05,1.383748e+06,2.230181e+05,2.550628e+05,7.001409e+05,1.149314e+06,4.747718e+06,...,2.406958e+06,3.714704e+06,2.439247e+07,1.432203e+06,285900.330459,7.194114e+06,1.049422e+06,512061.248627,4.853198e+05,269325.576000
4,OC99,OC,5.187022e+06,1.276135e+06,1.508248e+06,7.984348e+05,6.683302e+05,1.124542e+06,1.048222e+06,6.560021e+06,...,2.078472e+06,3.615789e+06,2.388205e+07,1.079829e+06,390051.083685,4.973265e+06,1.577625e+06,492468.444973,9.513722e+05,108986.286529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,CC32,other,2.259298e+06,1.343939e+06,1.072576e+06,8.836391e+05,4.082187e+05,1.398794e+06,1.224412e+06,9.584873e+06,...,2.644992e+06,3.991091e+06,3.397356e+07,2.343394e+06,393294.066629,7.231294e+06,1.278898e+06,716522.130687,1.084914e+06,222775.308015
94,BUT4,other,5.022872e+06,1.607385e+06,2.388110e+06,7.060213e+05,4.878218e+05,8.817753e+05,2.008881e+06,6.247535e+06,...,2.531400e+06,3.117078e+06,1.917957e+07,8.364534e+05,295906.740304,4.935578e+06,1.226907e+06,401686.341053,6.353400e+05,169655.083217
95,CC60,other,2.076435e+06,9.893924e+05,1.818200e+06,2.653812e+06,9.629553e+05,9.236050e+05,1.138840e+06,7.039278e+06,...,4.682950e+06,3.871834e+06,3.161519e+07,7.062074e+05,436682.506929,5.397743e+06,1.564098e+06,889081.835173,1.130176e+06,271239.648210
96,BOT34,other,1.280271e+06,5.303596e+05,8.241857e+05,1.297372e+06,4.153085e+05,6.268786e+05,6.073450e+05,6.587548e+06,...,2.559136e+06,3.942126e+06,2.275027e+07,1.037400e+06,388737.423271,5.796924e+06,1.240118e+06,371976.225433,5.624701e+05,239232.810651


In [40]:
dfgrp_test = df_test.filter(['group_other'], axis=1)
dfgroup_test = dfgrp_test['group_other'].map({'other': 0, 'OC': 1})

features_test= df_test.drop(['group_other','sample'],axis =1).columns  #get columns with feature names 
features_test= df_test[features_test]
features_test.astype(float);
x_test = features_test
x_test = (x_test- x_test.mean(axis=0))/x_test.std(axis=0) #autoscaled values for PCA
x_test.astype(float);
y_test=dfgroup_test.values

In [41]:

other_test = df_test[(df_test['group_other'] == 'other')]
OC_test = df_test[(df_test['group_other'] == 'OC')]

print("test other =", len(other_test))
print("test OC =", len(OC_test))


test other = 34
test OC = 64


### Classification metrics for imbalanced training set 

In [4]:
from sklearn.metrics import confusion_matrix
from sklearn import model_selection
from sklearn import model_selection
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold, cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report

In [43]:
def ml_model_2(X, y, model, kfold=10):
    
    '''
    FUNCTION: 
    A  machine learning function under Kfold cross validation conditions - used for training data 
    
    DEPENDENCY: 
    Various Sklearn packages
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import KFold
    from sklearn.metrics import roc_curve, auc
    from sklearn.metrics import roc_auc_score
    
    INPUTS: 
    X = numpy.ndaarray: features
    y = numpy.ndaarray: Labels encoded as 0s and 1s. 
    model = machine learning model
    kfold = numeric. K-fold default is 5. 
    
    OUTPUT: 
    A list containing the following machine learning metrics: 
    Area Under the Curve(AUC), Accuracy (ACC), Precision (PRE), 
    Recall (REC), F1-core (FS), Matthew's Correlation Coefficient (MCC). 
    
    '''

    cv = model_selection.StratifiedKFold(n_splits=kfold, random_state=42, shuffle=True)
    # define 'list' of various metrics
    AUC, ACC, REC, SPE = [], [], [], [] 
    performance = []
    
    for (train, test), i in zip(cv.split(X, y), range(kfold)):
        # training the model
        model.fit(X[train], y[train])
        y_pred = model.predict(X[test])
        
        #Retrieving metrics after testing
        modelprobs = model.predict_proba(X[test])
        fpr, tpr, thresholds = metrics.roc_curve(y[test], modelprobs[:,1])
        roc_auc = auc(fpr, tpr)
        accuracy = metrics.accuracy_score(y[test], y_pred)
        precision = metrics.precision_score(y[test], y_pred)
        recall = metrics.recall_score(y[test], y_pred)
        f1_score = metrics.f1_score(y[test], y_pred)
        mcc = metrics.matthews_corrcoef(y[test], y_pred)
        # coding specificity
        tn, fp, fn, tp = confusion_matrix(y[test], y_pred).ravel()
        specificity = tn / (tn+fp)
        # append metrics to list
        AUC.append(roc_auc)
        ACC.append(accuracy)
        #PRE.append(precision)
        REC.append(recall)
        #FS.append(f1_score)
        #MCC.append(mcc)
        SPE.append(specificity)
    return(mean(AUC).round(decimals=2), np.std(AUC).round(decimals=2)), (mean(ACC).round(decimals=2), np.std(ACC).round(decimals=2)),  (mean(REC).round(decimals=2), np.std(REC).round(decimals=2)),  (mean(SPE).round(decimals=2), np.std(SPE).round(decimals=2))


In [44]:
RFbase= RandomForestClassifier(random_state=42)
log_clf = LogisticRegression(random_state=42)
knn_clf = KNeighborsClassifier()
svm_clf = svm.SVC(probability=True)

#### Training 

In [45]:
metric_RFtrain = ml_model_2(RF_features_sel.to_numpy(), y, RFbase)
RF_AUC, RF_ACC, RF_REC, RF_SPE = metric_RFtrain[0], metric_RFtrain[1], metric_RFtrain[2], metric_RFtrain[3]

metric_LRtrain = ml_model_2(RF_features_sel.to_numpy(), y, log_clf)
LR_AUC, LR_ACC, LR_REC, LR_SPE = metric_LRtrain[0], metric_LRtrain[1], metric_LRtrain[2], metric_LRtrain[3]

metric_kNNtrain = ml_model_2(RF_features_sel.to_numpy(), y, knn_clf)
kNN_AUC, kNN_ACC, kNN_REC, kNN_SPE = metric_kNNtrain[0], metric_kNNtrain[1], metric_kNNtrain[2], metric_kNNtrain[3]


metric_svmtrain = ml_model_2(RF_features_sel.to_numpy(), y, svm_clf)
svm_AUC, svm_ACC, svm_REC, svm_SPE = metric_svmtrain[0], metric_svmtrain[1], metric_svmtrain[2], metric_svmtrain[3]


vclf = VotingClassifier(
    estimators=[('RF', RFbase),  
                ('lr', log_clf), ('knn', knn_clf), ('svm', svm_clf)],
    voting='soft')

metric_votingtrain = ml_model_2(RF_features_sel.to_numpy(), y, vclf) 
vclf_AUC, vclf_ACC, vclf_REC,  vclf_SPE = metric_votingtrain[0], metric_votingtrain[1], metric_votingtrain[2], metric_votingtrain[3]

In [46]:
Training = pd.DataFrame({'Random Forest' : metric_RFtrain,'Logistic Regression':(metric_LRtrain),'kNN':(metric_kNNtrain), 'SVM-lin': (metric_svmtrain), 'Voting': (metric_votingtrain)},
                   index = ['AUC','Accuracy','Sensitivity','SPE'])
Training 

,Random Forest,Logistic Regression,kNN,SVM-lin,Voting
AUC,"(0.72, 0.12)","(0.76, 0.1)","(0.67, 0.1)","(0.73, 0.12)","(0.74, 0.11)"
Accuracy,"(0.71, 0.07)","(0.72, 0.08)","(0.7, 0.07)","(0.72, 0.08)","(0.71, 0.07)"
Sensitivity,"(0.82, 0.08)","(0.84, 0.11)","(0.85, 0.08)","(0.89, 0.07)","(0.87, 0.07)"
SPE,"(0.52, 0.19)","(0.5, 0.18)","(0.46, 0.13)","(0.43, 0.19)","(0.44, 0.19)"


#### Test set

In [47]:
 #function for classtion metrics for test set
def ml_model(x_train, y_train, x_test, y_test, model):

    #cv = model_selection.StratifiedKFold(n_splits=kfold, random_state=42, shuffle=True)
    # define 'list' of various metrics
    AUC, ACC, REC, SPE = [], [], [], [] 
    performance = []
    
    
        # training the model
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
        #Retrieving metrics after testing
    modelprobs = model.predict_proba(x_test)
    #fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    fpr, tpr, thresholds = metrics.roc_curve(y_test,modelprobs[:,1] )
    roc_auc = auc(fpr, tpr)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    mcc = metrics.matthews_corrcoef(y_test, y_pred)
        # coding specificity
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
        # append metrics to list
    AUC.append(roc_auc)
    ACC.append(accuracy)
        #PRE.append(precision)
    REC.append(recall)
        #FS.append(f1_score)
        #MCC.append(mcc)
    SPE.append(specificity)
    return (mean(AUC).round(decimals=2), np.std(AUC).round(decimals=2)), (mean(ACC).round(decimals=2), np.std(ACC).round(decimals=2)),  (mean(REC).round(decimals=2), np.std(REC).round(decimals=2)),  (mean(SPE).round(decimals=2), np.std(SPE).round(decimals=2))


In [48]:
metric_RFtest = ml_model(RF_features_sel.to_numpy(), y, x_test[RF_features_sel_list], y_test, RFbase)
RF_AUC, RF_ACC, RF_REC, RF_SPE = metric_RFtest[0], metric_RFtest[1], metric_RFtest[2], metric_RFtest[3]

metric_LRtest = ml_model(RF_features_sel.to_numpy(), y,x_test[RF_features_sel_list], y_test, log_clf)
LR_AUC, LR_ACC, LR_REC, LR_SPE = metric_LRtest[0], metric_LRtest[1], metric_LRtest[2], metric_LRtest[3]

metric_kNNtest = ml_model(RF_features_sel.to_numpy(), y, x_test[RF_features_sel_list], y_test, knn_clf)
kNN_AUC, kNN_ACC, kNN_REC, kNN_SPE = metric_kNNtest[0], metric_kNNtest[1], metric_kNNtest[2], metric_kNNtest[3]


metric_svmtest = ml_model(RF_features_sel.to_numpy(), y, x_test[RF_features_sel_list], y_test, svm_clf)
svm_AUC, svm_ACC, svm_REC, svm_SPE = metric_svmtest[0], metric_svmtest[1], metric_svmtest[2], metric_svmtest[3]


vclf_test = VotingClassifier(
    estimators=[('RF', RFbase),  
                ('lr', log_clf), ('knn', knn_clf), ('svm', svm_clf)],
    voting='soft')

metric_votingtest = ml_model(RF_features_sel.to_numpy(), y, x_test[RF_features_sel_list], y_test,vclf) 
vclf_test_AUC, vclf_test_ACC, vclf_test_REC,  vclf_test_PE = metric_votingtest[0], metric_votingtest[1], metric_votingtest[2], metric_votingtest[3]

In [49]:
test = pd.DataFrame({'Random Forest' : metric_RFtest,'Logistic Regression':(metric_LRtest),'kNN':(metric_kNNtest), 'SVM-lin': (metric_svmtest), 'Voting': (metric_votingtest)},
                   index = ['AUC','Accuracy','Sensitivity','SPE'])
test  

,Random Forest,Logistic Regression,kNN,SVM-lin,Voting
AUC,"(0.75, 0.0)","(0.8, 0.0)","(0.7, 0.0)","(0.78, 0.0)","(0.77, 0.0)"
Accuracy,"(0.74, 0.0)","(0.76, 0.0)","(0.68, 0.0)","(0.73, 0.0)","(0.7, 0.0)"
Sensitivity,"(0.81, 0.0)","(0.86, 0.0)","(0.78, 0.0)","(0.86, 0.0)","(0.81, 0.0)"
SPE,"(0.62, 0.0)","(0.56, 0.0)","(0.5, 0.0)","(0.5, 0.0)","(0.5, 0.0)"


<a id="3"></a> 
### Balance training data with SMOTE 

In [5]:
import imblearn
import csv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import style
import scipy
from scipy import stats
from statsmodels.formula.api import ols
import statsmodels as sms
from statsmodels.stats import multitest
from scipy.stats import shapiro
from decimal import Decimal
from statistics import mean
from imblearn.over_sampling import SMOTE

In [51]:
sm = SMOTE(random_state=42) 
x_balanced, y_balanced = sm.fit_resample(x, y)

print('Number of non-oc samples',y_balanced.tolist().count(0))
print('OC samples', y_balanced.tolist().count(1))

Number of non-oc samples 144
OC samples 144


In [52]:
x_balanaced_scaled = (x_balanced-x_balanced.mean(axis=0))/x_balanced.std(axis=0)  

In [53]:
#dataframe of balanced training set with statistically sig features 
ML_x = x_balanaced_scaled[stat_sig_feat]
ML_x.astype(float);                                                                                               

<a id="4"></a> 
### Feature selection for balanced training set

In [54]:
RF_features_balanced = RF_selector(ML_x, y_balanced)
RF_features_balanced

,Annotations,Gini_Index
90,feat32Cer(d34:1)>Cer(d18:1/16:0)_P33,0.020804
7,feat134LPC(14:0)>LPC(14:0/0:0)_and_LPC(0:0/14:...,0.016823
92,feat331PC(O-36:5)_P332,0.016686
219,feat931PC(O-40:7)>PC(O-18:1/22:6)_and_PC(O-22:...,0.015501
160,feat58Cer(d43:2)>Cer(d18:1/25:1)_P59,0.014979
...,...,...
91,feat330PC(O-36:5)_P331,0.000861
209,feat909PC(O-36:6)>PC(O-16:1/20:5) M + CH3COOH,0.000723
10,feat142LPC(17:1)>LPC(17:1/0:0)_and_LPC(0:0/17:...,0.000620
5,feat129HexCer(d44:1-OH)>HexCer(d18:1/26:0-OH)_...,0.000610


In [55]:
RF_features_sel_bal = RF_features_balanced['Annotations'][:10].tolist()
RF_features_sel_bal = x_balanaced_scaled[RF_features_sel_bal]
RF_features_sel_list_bal = RF_features_sel_bal.columns
len(RF_features_sel_list_bal)

10

In [47]:
class_balanced = pd.DataFrame(y_balanced)
class_balanced.to_excel("class_10lipid_balanced_trainig.xlsx")

In [56]:
## Top 10 feature list
RF_features_sel_list_bal

Index(['feat32Cer(d34:1)>Cer(d18:1/16:0)_P33',
       'feat134LPC(14:0)>LPC(14:0/0:0)_and_LPC(0:0/14:0)_P135',
       'feat331PC(O-36:5)_P332',
       'feat931PC(O-40:7)>PC(O-18:1/22:6)_and_PC(O-22:7/18:0) M + CH3COOH',
       'feat58Cer(d43:2)>Cer(d18:1/25:1)_P59', 'feat390PE(O-38:4)_P391',
       'feat46Cer(d41:2)>Cer(d18:2/23:0)_P47',
       'feat426PS(38:4)>PS(18:0_20:4)_P427',
       'feat401PE(O-40:5)>PE(O-18:1/22:4)_P402', 'feat307PC(O-32:2)_P308'],
      dtype='object')

In [57]:
RF_features_top_10 = pd.DataFrame({'Features': RF_features_sel_list_bal})
#RF_features_top_10.to_excel(r'C:\Users\samyu\9_20_22 data\updated files 9_27_22\autoscaled training and test set\wo braca\RF selection 5_24_23\RF_features.xlsx', index = False)

#### Classification metrics for balanced training set

In [58]:
metric_RFtrain = ml_model_2(RF_features_sel_bal.to_numpy(), y_balanced, RFbase)
RF_AUC, RF_ACC, RF_REC, RF_SPE = metric_RFtrain[0], metric_RFtrain[1], metric_RFtrain[2], metric_RFtrain[3]

metric_LRtrain = ml_model_2(RF_features_sel_bal.to_numpy(), y_balanced, log_clf)
LR_AUC, LR_ACC, LR_REC, LR_SPE = metric_LRtrain[0], metric_LRtrain[1], metric_LRtrain[2], metric_LRtrain[3]

metric_kNNtrain = ml_model_2(RF_features_sel_bal.to_numpy(), y_balanced, knn_clf)
kNN_AUC, kNN_ACC, kNN_REC, kNN_SPE = metric_kNNtrain[0], metric_kNNtrain[1], metric_kNNtrain[2], metric_kNNtrain[3]


metric_svmtrain = ml_model_2(RF_features_sel_bal.to_numpy(), y_balanced, svm_clf)
svm_AUC, svm_ACC, svm_REC, svm_SPE = metric_svmtrain[0], metric_svmtrain[1], metric_svmtrain[2], metric_svmtrain[3]


vclf = VotingClassifier(
    estimators=[('RF', RFbase),  
                ('lr', log_clf), ('knn', knn_clf), ('svm', svm_clf)],
    voting='soft')

metric_votingtrain = ml_model_2(RF_features_sel_bal.to_numpy(), y_balanced, vclf) 
vclf_AUC, vclf_ACC, vclf_REC,  vclf_SPE = metric_votingtrain[0], metric_votingtrain[1], metric_votingtrain[2], metric_votingtrain[3]

In [59]:
Training_bal = pd.DataFrame({'Random Forest' : metric_RFtrain,'Logistic Regression':(metric_LRtrain),'kNN':(metric_kNNtrain), 'SVM-lin': (metric_svmtrain), 'Voting': (metric_votingtrain)},
                   index = ['AUC','Accuracy','Sensitivity','SPE'])
Training_bal

,Random Forest,Logistic Regression,kNN,SVM-lin,Voting
AUC,"(0.85, 0.07)","(0.83, 0.09)","(0.79, 0.1)","(0.83, 0.09)","(0.84, 0.09)"
Accuracy,"(0.77, 0.09)","(0.73, 0.1)","(0.72, 0.1)","(0.76, 0.09)","(0.75, 0.08)"
Sensitivity,"(0.78, 0.15)","(0.7, 0.12)","(0.67, 0.13)","(0.76, 0.09)","(0.72, 0.11)"
SPE,"(0.76, 0.12)","(0.75, 0.17)","(0.77, 0.14)","(0.77, 0.14)","(0.77, 0.14)"


#### Test set

In [60]:
metric_RFtest = ml_model(RF_features_sel_bal.to_numpy(), y_balanced, x_test[RF_features_sel_list_bal], y_test, RFbase)
RF_AUC, RF_ACC, RF_REC, RF_SPE = metric_RFtest[0], metric_RFtest[1], metric_RFtest[2], metric_RFtest[3]

metric_LRtest = ml_model(RF_features_sel_bal.to_numpy(), y_balanced,x_test[RF_features_sel_list_bal], y_test, log_clf)
LR_AUC, LR_ACC, LR_REC, LR_SPE = metric_LRtest[0], metric_LRtest[1], metric_LRtest[2], metric_LRtest[3]

metric_kNNtest = ml_model(RF_features_sel_bal.to_numpy(), y_balanced, x_test[RF_features_sel_list_bal], y_test, knn_clf)
kNN_AUC, kNN_ACC, kNN_REC, kNN_SPE = metric_kNNtest[0], metric_kNNtest[1], metric_kNNtest[2], metric_kNNtest[3]


metric_svmtest = ml_model(RF_features_sel_bal.to_numpy(), y_balanced, x_test[RF_features_sel_list_bal], y_test, svm_clf)
svm_AUC, svm_ACC, svm_REC, svm_SPE = metric_svmtest[0], metric_svmtest[1], metric_svmtest[2], metric_svmtest[3]


vclf_test = VotingClassifier(
    estimators=[('RF', RFbase),  
                ('lr', log_clf), ('knn', knn_clf), ('svm', svm_clf)],
    voting='soft')

metric_votingtest = ml_model(RF_features_sel_bal.to_numpy(), y_balanced, x_test[RF_features_sel_list_bal], y_test,vclf) 
vclf_test_AUC, vclf_test_ACC, vclf_test_REC,  vclf_test_PE = metric_votingtest[0], metric_votingtest[1], metric_votingtest[2], metric_votingtest[3]

In [61]:
test_ = pd.DataFrame({'Random Forest' : metric_RFtest,'Logistic Regression':(metric_LRtest),'kNN':(metric_kNNtest), 'SVM-lin': (metric_svmtest), 'Voting': (metric_votingtest)},
                   index = ['AUC','Accuracy','Sensitivity','SPE'])
test_ 

,Random Forest,Logistic Regression,kNN,SVM-lin,Voting
AUC,"(0.82, 0.0)","(0.8, 0.0)","(0.77, 0.0)","(0.82, 0.0)","(0.83, 0.0)"
Accuracy,"(0.78, 0.0)","(0.64, 0.0)","(0.69, 0.0)","(0.74, 0.0)","(0.74, 0.0)"
Sensitivity,"(0.78, 0.0)","(0.56, 0.0)","(0.62, 0.0)","(0.72, 0.0)","(0.72, 0.0)"
SPE,"(0.76, 0.0)","(0.79, 0.0)","(0.82, 0.0)","(0.79, 0.0)","(0.79, 0.0)"


In [64]:
test_10lipid = pd.DataFrame(x_test[RF_features_sel_list_bal])
test_10lipid.to_excel('test_set_10lipid.xlsx')

class_10lipid = pd.DataFrame(y_test)
class_10lipid.to_excel('class-10lipid.xlsx')

<a id="5"></a> 
### Early stage samples from test set

In [35]:
cd C:\Users\samyu\9_20_22 data\updated files 9_27_22\autoscaled training and test set\wo braca\RF selection 5_24_23

C:\Users\samyu\9_20_22 data\updated files 9_27_22\autoscaled training and test set\wo braca\RF selection 5_24_23


In [62]:
df_early = pd.read_excel('early test set.xlsx', sheet_name = 'early test')
df_early.head()

,sample,group_other,feat32Cer(d34:1)>Cer(d18:1/16:0)_P33,feat134LPC(14:0)>LPC(14:0/0:0)_and_LPC(0:0/14:0)_P135,feat331PC(O-36:5)_P332,feat931PC(O-40:7)>PC(O-18:1/22:6)_and_PC(O-22:7/18:0) M + CH3COOH,feat58Cer(d43:2)>Cer(d18:1/25:1)_P59,feat390PE(O-38:4)_P391,feat46Cer(d41:2)>Cer(d18:2/23:0)_P47,feat426PS(38:4)>PS(18:0_20:4)_P427,feat401PE(O-40:5)>PE(O-18:1/22:4)_P402,feat307PC(O-32:2)_P308
0,OC40,OC,3.498908e+06,3.689594e+06,3.951554e+05,31575.366967,1.220290e+06,2.352056e+06,1.594191e+06,1.003055e+06,623464.986626,2.459390e+07
1,OC65,OC,2.101819e+06,2.899069e+06,2.608490e+05,73480.033466,7.694926e+05,1.516487e+06,6.757231e+05,1.250694e+06,293686.529330,1.257053e+07
2,OC99,OC,2.376335e+06,2.631487e+06,1.428433e+06,196710.858101,6.091204e+05,5.147528e+06,7.003833e+05,3.446328e+06,118876.463556,2.681342e+07
3,OC4,OC,2.273932e+06,3.183964e+06,3.891462e+05,114635.091455,1.132670e+06,1.815802e+06,1.122377e+06,1.973652e+06,373069.036429,1.688152e+07
4,OC86,OC,2.738955e+06,2.133532e+06,3.349854e+05,219085.385442,7.855886e+05,2.561231e+06,6.611007e+05,2.654182e+06,331237.966657,2.070069e+07


In [63]:
#assign class and features 
other_early = df_early[(df_early['group_other'] == 'other')]
OC_early = df_early[(df_early['group_other'] == 'OC')]

#list of features
features_early = df_early.drop(['sample','group_other'],axis = 1).columns
features_df_early = df_early.drop(['sample','group_other'],axis = 1)

In [64]:
dfgrp_early = df_early.filter(['group_other'], axis=1)
dfgroup_early = dfgrp_early['group_other'].map({'other': 0, 'OC': 1})
x_early= features_df_early #df with stat sig features
y_early= dfgroup_early.values #class as 0 or 1 for other and OC 

x_scaled_early = (x_early-x_early.mean(axis=0))/x_early.std(axis=0)



In [65]:
metric_RFtest = ml_model(RF_features_sel_bal.to_numpy(), y_balanced, x_scaled_early, y_early, RFbase)
RF_AUC, RF_ACC, RF_REC, RF_SPE = metric_RFtest[0], metric_RFtest[1], metric_RFtest[2], metric_RFtest[3]

metric_LRtest = ml_model(RF_features_sel_bal.to_numpy(), y_balanced, x_scaled_early, y_early, log_clf)
LR_AUC, LR_ACC, LR_REC, LR_SPE = metric_LRtest[0], metric_LRtest[1], metric_LRtest[2], metric_LRtest[3]

metric_kNNtest = ml_model(RF_features_sel_bal.to_numpy(), y_balanced, x_scaled_early, y_early, knn_clf)
kNN_AUC, kNN_ACC, kNN_REC, kNN_SPE = metric_kNNtest[0], metric_kNNtest[1], metric_kNNtest[2], metric_kNNtest[3]


metric_svmtest = ml_model(RF_features_sel_bal.to_numpy(), y_balanced, x_scaled_early, y_early, svm_clf)
svm_AUC, svm_ACC, svm_REC, svm_SPE = metric_svmtest[0], metric_svmtest[1], metric_svmtest[2], metric_svmtest[3]


vclf_early = VotingClassifier(
    estimators=[('RF', RFbase),  
                ('lr', log_clf), ('knn', knn_clf), ('svm', svm_clf)],
    voting='soft')

metric_votingtest = ml_model(RF_features_sel_bal.to_numpy(), y_balanced, x_scaled_early, y_early,vclf_early) 
vclf_test_AUC, vclf_test_ACC, vclf_test_REC,  vclf_test_PE = metric_votingtest[0], metric_votingtest[1], metric_votingtest[2], metric_votingtest[3]

In [1]:
test_early = pd.DataFrame({'Random Forest' : metric_RFtest,'Logistic Regression':(metric_LRtest),'kNN':(metric_kNNtest), 'SVM-lin': (metric_svmtest), 'Voting': (metric_votingtest)},
                   index = ['AUC','Accuracy','Sensitivity','SPE'])
test_early 

NameError: name 'pd' is not defined